In [ ]:
%load_ext autoreload
%autoreload 2
 
%load_ext magic_duckdb
from magic_duckdb.extras import logging_init

logging_init.log_to_file("magic_duckdb2.log")
#%config Completer.use_jedi = False
#%config IPCompleter.greedy=True

from IPython.core.getipython import get_ipython
ip=get_ipython()
#ip.Completer.suppress_competing_matchers=False

In [ ]:

import duckdb
import numpy as np
from magic_duckdb import magic
from magic_duckdb.autocomplete.autocompletion_v2 import DqlCustomCompleter
from pandas import DataFrame
from types import SimpleNamespace
from IPython.core.getipython import get_ipython

con = duckdb.connect()


In [ ]:
# Test of underlying v2 completer (ipython 8.6.0 or above)

some_tablename = "tablenamereallylong"
# create some tables
simpledf = DataFrame(np.random.randn(10, 20))  # type: ignore # noqa

con.sql(f"CREATE OR REPLACE TABLE {some_tablename} as select * from simpledf")
con.sql("CREATE OR REPLACE TABLE  sometablename2 as select * from range(10) t(my_column_1)")
con.sql(
    "CREATE OR REPLACE TABLE  longtablenameishardtomakeup as select * from range(10) t(d)"
)
con.sql("show tables").df()

# test autocompletion
magic.connection = con
completer = DqlCustomCompleter(shell=get_ipython())

# completer finds the table names
event = SimpleNamespace(
    full_text="%dql s", token="s"
)
r = completer.line_completer(event)
results = [sc.text for sc in r["completions"]]
assert results is not None
assert "SELECT" in results

event = SimpleNamespace(
    token="from", full_text="%dql select * from ",
)
r = completer.line_completer(event)
results = [sc.text for sc in r["completions"]]
assert results is not None
assert some_tablename in results
assert "SELECT" not in results

# Tablename doesn't exist
event = SimpleNamespace(
    token="blah.", full_text="%dql select blah.",
)
r = completer.line_completer(event)
results = [sc.text for sc in r["completions"]]
assert results is not None
assert len(results) == 0

event = SimpleNamespace(
    token="tablenamereallylong.", full_text="%dql  tablenamereallylong.",
)
r = completer.line_completer(event)
results = [sc.text for sc in r["completions"]]

assert results is not None
assert "19" in results

event = SimpleNamespace(
    token="tablenamereallylong.", full_text="%dql select tablenamereallylong.",
)
r = completer.line_completer(event)
results = [sc.text for sc in r["completions"]]
assert results is not None
assert "17" in results

event = SimpleNamespace(
    token=f"{some_tablename}.", full_text=f"%dql select {some_tablename}.",
)
r = completer.line_completer(event)
results = [sc.text for sc in r["completions"]]
assert results is not None
assert "19" in results
# Tablename doesn't exist


In [ ]:
# Test using IPCompleter
# This uses the full IPCompleter stack, so is more realistic in terms of results

from IPython.core.getipython import get_ipython

ip = get_ipython()
event = SimpleNamespace(line='%dql get_ipython', symbol='g', command='%dql get_ipython', text_until_cursor=f'%dql get_ipython', cursor_pos=6)

r = ip.Completer.complete(line_buffer="%dql crea")
assert "CREATE TABLE" in r[1]

r = ip.Completer.complete(line_buffer="%dql sometablename2.")
assert "my_column_1" in r[1]

r = ip.Completer.complete(line_buffer="%dql select * from ")
assert "sometablename2" in r[1]

In [ ]:
%dql CREATE TABLE IF NOT EXISTS my_new_table as select 'abc' as my_first_column, 'def' as my_second_column
%dql CREATE TABLE IF NOT EXISTS another_table as select 'abc' as my_first_column, 'def' as my_second_column

someDataFrame = DataFrame(columns=["col123", "col345", "col919191"])
